<a href="https://colab.research.google.com/github/6Santiago9/Entregas/blob/main/03_Entrenamiento_con_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RandomForest
En el primer entrenamiento use este modelo con la misma limpieza y preprosesado que hice en la entrega 2 y mirar cuando me daba

# Importación de Librerías

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Configuración Kaggle

In [ ]:
# Establecer tus credenciales de Kaggle
os.environ['KAGGLE_USERNAME'] = 'cesarmartinezia'
os.environ['KAGGLE_KEY'] = '3bd3c2a5994356c24295cd5c6d8bba59'

In [ ]:
# Crear manualmente el archivo kaggle.json a partir de esas variables
!mkdir -p ~/.kaggle
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (os.environ['KAGGLE_USERNAME'], os.environ['KAGGLE_KEY']))

In [ ]:
# Ajustar permisos
!chmod 600 /root/.kaggle/kaggle.json

# Descarga de Dataset

In [ ]:
# Descargar los datos de la competencia
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 2.73GB/s]


In [ ]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


# Cargar el train y test

In [ ]:
df = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
testID = test.copy()

In [ ]:
# Tamaño del dataset
print(df.shape, test.shape)

(692500, 21) (296786, 20)


# Limpieza tanto en test como train

In [ ]:
# Revisar valores faltantes
df.isnull().sum().sort_values(ascending=False).head(10)
test.isnull().sum().sort_values(ascending=False).head(10)
# Si es categórica reemplazar con la moda (valor más frecuente)
for col in df.select_dtypes(include=['object']):
    df[col].fillna(df[col].mode()[0], inplace=True)
for col in test.select_dtypes(include=['object']):
    test[col].fillna(test[col].mode()[0], inplace=True)

/tmp/ipython-input-4292607415.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
/tmp/ipython-input-4292607415.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

## Columnas duplicadas

In [ ]:
df.drop('F_TIENEINTERNET.1', axis=1, inplace=True)
test.drop('F_TIENEINTERNET.1', axis=1, inplace=True)

## Palabras mal escritas o con mucha similitud


In [ ]:
# Función para eliminar tildes y normalizar texto
def normalizar_texto(texto):
    if isinstance(texto, str):
        # Quitar tildes (por ejemplo: Á → A)
        texto = ''.join(
            c for c in unicodedata.normalize('NFD', texto)
            if unicodedata.category(c) != 'Mn'
        )
        # Convertir a mayúsculas
        texto = texto.strip().upper()
        return texto
    else:
        return texto
# Aplicar la función directamente sobre la columna original
df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].apply(normalizar_texto)
test['E_PRGM_ACADEMICO'] = test['E_PRGM_ACADEMICO'].apply(normalizar_texto)
# Reemplazamos el nombre de algunas variables
df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].str.replace('DISENO','DISEÑO', regex=False)
test["E_PRGM_ACADEMICO"] = test["E_PRGM_ACADEMICO"].str.replace('DISENO','DISEÑO', regex=False)
df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].str.replace('LICENCIATURA M¿SICA','LICENCIATURA', regex=False)
test["E_PRGM_ACADEMICO"] = test["E_PRGM_ACADEMICO"].str.replace('LICENCIATURA M¿SICA','LICENCIATURA', regex=False)

Eliminar espacios

In [ ]:
df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()) if isinstance(x, str) else x)
test['E_PRGM_ACADEMICO'] = test['E_PRGM_ACADEMICO'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()) if isinstance(x, str) else x)

Ahora unimos todos los programas que tienen la misma similitud

In [ ]:
pip install thefuzz[speedup]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.6 MB/s eta 0:00:00


In [ ]:
from thefuzz import process
# -- Selecciona la columna que quieres agrupar ---
columna = 'E_PRGM_ACADEMICO'
# -- Define las categorías limpias (referencias) ---
categorias_referencia = [
    'ADMINISTRACION DE EMPRESAS',
    'ADMINISTRACION',
    'COMERCIO',
    'ARTES',
    'BIOLOGIA',
    'CIENCIAS',
    "CINE",
    "COMUNICACION",
    'CONTADURIA',
    "DERECHO",
    "DISENO",
    "INGENIERIA DE SISTEMAS",
    "INGENIERIA CIVIL",
    "INGENIERIA DE MINAS",
    "INGENIERIA ELECTRICA",
    "INGENIERIA INDUSTRIAL",
    "INGENIERIA MECANICA",
    "INGENIERIA QUIMICA",
    "MERCADEO",
    "LICENCIATURA",
    "NEGOCIOS INTERNACIONALES",
    "QUIMICA",
    "GASTRONOMIA"
]
#  Aplica el agrupamiento difuso
def corregir_categoria(valor):
    if pd.isnull(valor):
        return valor
    mejor_match = process.extractOne(valor, categorias_referencia)
    if mejor_match[1] >= 80:  # porcentaje mínimo de similitud
        return mejor_match[0]
    else:
        return valor
df[columna] = df[columna].apply(corregir_categoria)
test[columna] = test[columna].apply(corregir_categoria)

Ya despues de esto miramos que algunas carreras tienen que ver con lo mismo pero tenian diferente nombre por lo que se procede a agruparlas en una sola

In [ ]:
# Diccionario con las agrupaciones
agrupamientos = {
    'ADMINISTRACION DE EMPRESAS': ['ADMINISTRACION'],
    "LICENCIATURA": ['LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES: LENGUA CASTELLANA E IDIOMA EXTRANJERO: INGLES'],
    "DERECHO": ["JURISPRUDENCIA"],
    "PSICOLOGIA": ["PSICOLOGA", "PROFESIONAL EN PSICOLOGIA"],
    "INGENIERIA DE SISTEMAS": ["INGENIERIA DE SOFTWARE", "INGENIERIA EN SOFTWARE", "INGENIER¿A EN SOFTWARE", "INGENIIERIA DE SOFTWARE"],
    "TRABAJO SOCIAL": ["PROFESIONAL EN TRABAJO SOCIAL"],
    "DISENO": ["DISE¿O CROSSMEDIA", "PROFESIONAL EN DISEÑO GRAFICO", "PROFESIONAL EN DISEÑO INDUSTRIAL", "PROFESIONAL EN DISEÑO VISUAL", "PROFESIONAL UNIVERSITARIO DISEÑO GRAFICO DIGITAL", "PROFESIONAL UNIVERSITARIO EN DISEÑO TEXTIL PARA LA INDUSTRIA", "PROFESIONAL UNIVERSITARIO EN FOTOGRAFIA PARA MEDIOS"],
    "ECONOMIA": ["ECONOMIA Y FINANZAS INTERNACIONALES", "ECONOM¿A"],
    "MEDICINA VETERINARIA": ["MEDICINA VETERINARIA Y ZOOTECNIA", "ZOOTECNIA"],
    "MERCADEO": ["MARKETING", "MARKETING Y LOGISTICA", "PROFESIONAL EN MARKETING INTERNACIONAL"],
    "ARTES": ["ARTES PLASTICAS CON ENFASIS EN PROCESOS BIDIMENSIONALES Y TRIDIMENSIONALES"],
    "SEGURIDAD Y SALUD EN EL TRABAJO": ["PROFESIONAL EN SEGURIDAD Y SALUD EN EL TRABAJO", "SALUD OCUPACIONAL"],
    "QUIMICA": ["QU¿MICA FARMAC¿UTICA"],
    "LENGUAS MODERNAS": ["LENGUAS MODERNAS Y CULTURA", "LENGUAS Y CULTURA", "LETRAS - FILOLOGIA HISPANICA", "PROFESIONAL EN LENGUAS MODERNAS", "PROGRAMA PROFESIONAL UNIVERSITARIO EN LENGUAS EXTRANJERAS", "TRADUCCION INGLES-FRANCES-ESPANOL"],
    "INSTRUMENTACION QUIRURGICA": ["INSTRUMENTACION QUIRURGICA PROFESIONAL", "INTRUMENTACION QUIRURGICA"],
    "PUBLICIDAD": ["PUBLICIDAD Y MARKETING CREATIVO", "PUBLICIDAD Y MARKETING DIGITAL", "PUBLICIDAD INTERNACIONAL"],
    "MUSICA": ["MUSICA INSTRUMENTAL", "MUSICA INSTRUMENTO", "MUSICA- CANTO", "DIRECCION DE BANDA", "INTERPRETACION MUSICAL", "MAESTRO EN MUSICA", "TEATRO MUSICAL", "FORMACION MUSICAL"],
    "AGRONOMIA": ["AGRONOMIA DEL TROPICO HUMEDO"],
    "FILOSOFIA": ["FILOSOFIA Y HUMANIDADES", "FILOSOFIA Y LETRAS", "LETRAS - FILOLOGIA HISPANICA"],
    "ACTIVIDAD FISICA Y DEPORTE": ["CULTURA FISICA Y DEPORTE", "CULTURA FISICA, DEPORTE Y RECREACION", "DEPORTE", "DEPORTE Y ACTIVIDAD FISICA", "DEPORTE Y ACTIVIDADA FISICA", "DEPORTE Y CULTURA FISICA", "ENTRENAMIENTO DEPORTIVO", "PROFESIONAL EN ACTIVIDAD FISICA Y DEPORTE",
                                   "PROFESIONAL EN DEPORTE", "PROFESIONAL EN ENTRENAMIENTO DEPORTIVO", "FORMACION PROFESIONAL EN DEPORTE", "GESTI¿N DEPORTIVA", "RECREACION"],
    "GEOGRAFIA": ["GEOGRAFIA APLICADA A LA ORGANIZACION DEL ESPACIO Y PLANIFICACION REGIONAL", "GEOGRAFIA DEL DESARROLLO REGIONAL Y AMBIENTAL", "GEOLOGIA", "GEOLOG¿A", "URBANISMO"],
    "CONSTRUCCION": ["CONSTRUCCION Y GESTION EN ARQUITECTURA", "CONSTRUCCIONES CIVILES"],
    "HISTORIA": ["HISTORIA Y ARCHIVISTICA", "HISTORIA Y PATRIMONIO"],
    "LITERATURA": ["LINGUISTICA Y LITERATURA", "ESTUDIOS LITERARIOS", "ESTUDIOS LITERARIOS Y EDICION", "CREACION LITERARIA"],
    "CRIMINALISTICA": ["INVESTIGACION CRIMINAL", "PROFESIONAL EN INVESTIGACION CRIMINAL", "PROFESIONAL EN CRIMINALISTICA"],
    "INGENIERIA DE CONTROL": ["INGENIERIA EN CONTROL"],
    "INGENIERIA EN ENERGIA": ["INGENIERIA EN ENERGIAS"],
    "MATEMATICAS": ["MATEMATICA APLICADA", "MATEMATICAS APLICADAS", "MATEMATICAS CON ENFASIS EN ESTADISTICA"] ,
    "GESTION CULTURAL": ["GESTION CULTURAL Y COMUNICATIVA", "GESTION EMPRESARIAL", "GESTION EN ECOLOGIA Y TURISMO", "GESTION Y DESARROLLO URBANOS", "GOBIERNO Y ASUNTOS PUBLICOS"],
    "HOTELERIA Y TURISMO": ["HOTELERIA Y TURISMO ECOLOGICO"],

}
# Función para reemplazar dentro del mismo df
def normalizar_programa(prog):
    for clave, valores in agrupamientos.items():
        if prog in valores:
            return clave
    return prog  # si no está en el diccionario, se deja igual
#  Aplicar directamente al df original
df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].apply(normalizar_programa)
test['E_PRGM_ACADEMICO'] = test['E_PRGM_ACADEMICO'].apply(normalizar_programa)

Evalumos que programas son los mas escogidos para que esto mejore el estudio, y juntamos en un solo subnivel las carreras que no son tan escogidas, como en el test son menos datos el umbral me toco cambiarlo, ya que al ejecutarlo me daba diferentes carreras

In [ ]:
# Paso 1: Contar la frecuencia de cada carrera
frecuencias = df['E_PRGM_ACADEMICO'].value_counts()
# Paso 2: Definir el umbral
umbral = 10000
# Paso 3: Identificar carreras poco escogidas
carreras_poco_frecuentes = frecuencias[frecuencias < umbral].index
# Paso 4: Reemplazar las carreras poco escogidas por "OTRAS"
df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].replace(carreras_poco_frecuentes, 'OTRAS')

In [ ]:
frecuencia = test['E_PRGM_ACADEMICO'].value_counts()
umbrall = 4349
carreras_poco_frecuentes = frecuencia[frecuencia < umbrall].index
test['E_PRGM_ACADEMICO'] = test['E_PRGM_ACADEMICO'].replace(carreras_poco_frecuentes, 'OTRAS')

# Preprosesado

Agrupamos las variables  nosabe, no aplica y las unimos a ninguna en las columnas de "educacion padre y madre", esto con el fin de hacer que los datos se miren mejor y no tengamos muchos subniveles que no aportan nada

In [ ]:
df.F_EDUCACIONMADRE = ['Ninguno' if i in ['No sabe', 'No Aplica'] else i for i in df.F_EDUCACIONMADRE.values]
df.F_EDUCACIONPADRE = ['Ninguno' if i in ['No sabe', 'No Aplica'] else i for i in df.F_EDUCACIONPADRE.values]
test.F_EDUCACIONMADRE = ['Ninguno' if i in ['No sabe', 'No Aplica'] else i for i in test.F_EDUCACIONMADRE.values]
test.F_EDUCACIONPADRE = ['Ninguno' if i in ['No sabe', 'No Aplica'] else i for i in test.F_EDUCACIONPADRE.values]

## Convertir variables categoricas en numericas

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['F_TIENELAVADORA'] = le.fit_transform(df['F_TIENELAVADORA']) # Aqui el 1 es si y 0 es no
df['F_TIENEINTERNET'] = le.fit_transform(df['F_TIENEINTERNET']) # Aqui el 1 es si y 0 es no
df['F_TIENEAUTOMOVIL'] = le.fit_transform(df['F_TIENEAUTOMOVIL']) # Aqui el 1 es si y 0 es no
df['E_PRIVADO_LIBERTAD'] = le.fit_transform(df['E_PRIVADO_LIBERTAD']) # Aqui el 1 es si y 0 es no
df['E_PAGOMATRICULAPROPIO'] = le.fit_transform(df['E_PAGOMATRICULAPROPIO']) # Aqui el 1 es si y 0 es no
df['F_TIENECOMPUTADOR'] = le.fit_transform(df['F_TIENECOMPUTADOR']) # Aqui el 1 es s y 0 es n
df['E_PRIVADO_LIBERTAD'] = le.fit_transform(df['E_PRIVADO_LIBERTAD']) # Aqui el 1 es si y 0 es no
test['F_TIENELAVADORA'] = le.fit_transform(test['F_TIENELAVADORA']) # Aqui el 1 es si y 0 es no
test['F_TIENEINTERNET'] = le.fit_transform(test['F_TIENEINTERNET']) # Aqui el 1 es si y 0 es no
test['F_TIENEAUTOMOVIL'] = le.fit_transform(test['F_TIENEAUTOMOVIL']) # Aqui el 1 es si y 0 es no
test['E_PRIVADO_LIBERTAD'] = le.fit_transform(test['E_PRIVADO_LIBERTAD']) # Aqui el 1 es si y 0 es no
test['E_PAGOMATRICULAPROPIO'] = le.fit_transform(test['E_PAGOMATRICULAPROPIO']) # Aqui el 1 es si y 0 es no
test['F_TIENECOMPUTADOR'] = le.fit_transform(test['F_TIENECOMPUTADOR']) # Aqui el 1 es s y 0 es n
test['E_PRIVADO_LIBERTAD'] = le.fit_transform(test['E_PRIVADO_LIBERTAD']) # Aqui el 1 es si y 0 es no

Aqui reemplazamos variables categoricas que contienen mas de un nivel

In [ ]:
# En estrato solo se puso el numero al que pertenecian.
estrato = {'Sin Estrato' : 0,
    'Estrato 1' : 1,
    'Estrato 2' : 2,
    'Estrato 3' : 3,
    'Estrato 4' : 4,
    'Estrato 5' : 5,
    'Estrato 6' : 6}
# En trabaja se puso un valor que estara en el rango de la descripción dada
trabaja = {"Más de 30 horas" : 31,
           "0" : 0,
           "Entre 11 y 20 horas" : 15,
           "Entre 21 y 30 horas" : 25,
           "Menos de 10 horas" : 5}
# En matricula tambien se puso un valor que estara entre el valor menconado.
matricula = {"Entre 1 millón y menos de 2.5 millones" : 2,
             "Entre 2.5 millones y menos de 4 millones" : 3,
             "Menos de 500 mil" : 1,
             "Entre 500 mil y menos de 1 millón" : 7,
             "Entre 4 millones y menos de 5.5 millones" : 5,
             "Más de 7 millones" : 8,
             "Entre 5.5 millones y menos de 7 millones" : 6,
             "No pagó matrícula" : 0}
df['E_VALORMATRICULAUNIVERSIDAD'].replace(matricula, inplace=True)
df['E_HORASSEMANATRABAJA'].replace(trabaja, inplace=True)
df['F_ESTRATOVIVIENDA'].replace(estrato, inplace=True)
test['E_VALORMATRICULAUNIVERSIDAD'].replace(matricula, inplace=True)
test['E_HORASSEMANATRABAJA'].replace(trabaja, inplace=True)
test['F_ESTRATOVIVIENDA'].replace(estrato, inplace=True)

/tmp/ipython-input-1182468233.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['E_VALORMATRICULAUNIVERSIDAD'].replace(matricula, inplace=True)
/tmp/ipython-input-1182468233.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['E_VALORMATRICULAUNIVERSIDAD'].replace(matricula, inplace=True)


## Utilizar One-HOt

In [ ]:
df = df.copy()
df = pd.get_dummies(df, columns=['E_PRGM_DEPARTAMENTO'], dtype=int)
df = pd.get_dummies(df, columns=['F_EDUCACIONMADRE'], dtype=int)
df = pd.get_dummies(df, columns=['F_EDUCACIONPADRE'], dtype=int)
df = pd.get_dummies(df, columns=['E_PRGM_ACADEMICO'], dtype=int)
test = test.copy()
test = pd.get_dummies(test, columns=['E_PRGM_DEPARTAMENTO'], dtype=int)
test = pd.get_dummies(test, columns=['F_EDUCACIONMADRE'], dtype=int)
test = pd.get_dummies(test, columns=['F_EDUCACIONPADRE'], dtype=int)
test = pd.get_dummies(test, columns=['E_PRGM_ACADEMICO'], dtype=int)

## Normalizacion de variables numericas

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
num_cols = test.select_dtypes(include=['float64', 'int64']).columns
df[num_cols] = scaler.fit_transform(df[num_cols])
test[num_cols] = scaler.transform(test[num_cols])

In [ ]:
train = df.copy()
test = test.copy()

## Entrenamiento

In [ ]:
# 1. Separar variable objetivo
X = train.drop(["RENDIMIENTO_GLOBAL"], axis=1)
y = train["RENDIMIENTO_GLOBAL"]

test_IDs = testID['ID']
X_test = testID.drop(columns=['ID'])
# El test no tiene la variable objetivo (como debe ser)
X_test = test.copy()

#
# 2. Dividir en train/validación
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train:", X_train.shape, "Valid:", X_valid.shape)

# 3. Modelo Random Forest optimizado
# A la hora de ejecutarlo se demoraba mucho y me daba error de RAm por eso me toco minimizar algunas cosas
# - menos árboles
# - profundidad limitada
# - usar solo 70% del dataset por árbol
# - usa todos los núcleos (n_jobs=-1)

rf = RandomForestClassifier(
    n_estimators=200,     # 200 árboles (suficiente)
    max_depth=25,         # evita árboles gigantes
    max_samples=0.7,     # usa 70% de datos por árbol
    n_jobs=-1,            # usa todos los núcleos
    random_state=42
)

print("Entrenando Random Forest optimizado...")
rf.fit(X_train, y_train)

# 4. Evaluación en validación
y_pred_valid = rf.predict(X_valid)
acc = accuracy_score(y_valid, y_pred_valid)

print("📌 Accuracy en validación:", acc)

# 5. Predicciones para Kaggle
preds_test = rf.predict(X_test)

# 6. Crear submission.csv
submission = pd.DataFrame({
    "ID": test_IDs,
    "RENDIMIENTO_GLOBAL": preds_test
})

submission.to_csv("submission_rf.csv", index=False)

print("📁 Archivo 'submission_rf.csv' creado exitosamente.")
print(submission.head())

Train: (554000, 85) Valid: (138500, 85)
Entrenando Random Forest optimizado...
📌 Accuracy en validación: 0.41028880866425993
📁 Archivo 'submission_rf.csv' creado exitosamente.
       ID RENDIMIENTO_GLOBAL
0  550236               alto
1   98545         medio-bajo
2  499179               alto
3  782980               bajo
4  785185               bajo
